# Профилирование модели

Одним из первых шагов для анализа вычислительной сложности модели является ее профилирование.

Этот пример лучше запускать либо на компьютере с GPU от NVidia, либо в [Google Colab](https://colab.research.google.com/).

In [ ]:
!pip3 install torch torchvision

In [ ]:
import torch

# Let's make sure GPU is available!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
from torchvision import models

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.to(device)
model.eval()

named_layers = list(dict(model.named_modules()).keys())
print('Layers:')
for l in named_layers:
    print(l)

In [ ]:
from torch.profiler import profile, ProfilerActivity, record_function

input = torch.randn(1, 3, 224, 224).to(device)

# Warm-up
model(input)

activities = [ProfilerActivity.CPU]
if torch.cuda.is_available():
    activities += [ProfilerActivity.CUDA]

with profile(activities=activities, profile_memory=True, record_shapes=True, acc_events=True) as prof:
    with record_function("model_inference"):
        output = model(input)

sort_key = device.type + "_time_total"

print(prof.key_averages().table(sort_by=sort_key))

prof.export_chrome_trace("trace.json")